In [ ]:
!pip install langchain
!pip install langchain-core
!pip install langchain-expiremental
!pip install langchain-community
!pip install crewai
!pip install crewai[tools]

ERROR: Could not find a version that satisfies the requirement langchain-expiremental (from versions: none)
ERROR: No matching distribution found for langchain-expiremental
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.24
    Uninstalling langchain-core-0.3.24:
      Successfully uninstalled langchain-core-0.3.24
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.11
    Uninstalling la

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import PDFSearchTool

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
rag_tool = PDFSearchTool(pdf='/content/What-Is-Diabetes.pdf')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.34it/s]


In [ ]:
rag_tool.run("What is Diabetes?")

Using Tool: Search a PDF's content


'Relevant Content:\n1 Feeling Tired Thirsty Frequent urination Hungry Unexplained weight loss Very dry skin Blurry vision “Pins and needles” feeling in feet Sores that do not heal Feeling irritable The symptoms of type 2 diabetes develop over time. Some people have symptoms and others do not. What Are The Symptoms? There are two types of diabetes: • Type 1 diabetes happens when the body stops making insulin. • Type 2 diabetes occurs when some insulin is made, but the body cannot use it well. Type 2 diabetes is more common. Diabetes is a serious disease. It happens when the body does not make enough insulin or cannot use it well. This causes high levels of glucose (blood sugar) to build up in the blood. As a result, the body does not work like it should. • The food we eat is digested in our stomach. Here it is turned into glucose, or blood sugar. • Blood takes glucose to our cells. • Glucose must enter into our cells so that we can have energy. Gluose needs the help of insulin to do thi

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['TAVILY_API_KEY'] = ''
web_search_tool = TavilySearchResults(k = 1)

In [ ]:
from crewai_tools  import tool
@tool
def router_tool(question):
  """Router Function"""
  if 'Diabetes' in question:
    return 'vectorstore'
  else:
    return 'web_search'

In [ ]:
web_search_tool.run("What is Diabetes?")

[{'url': 'https://www.cdc.gov/diabetes/about/index.html',
  'content': 'Diabetes is a condition that affects how your body uses insulin and blood sugar. Learn about the three main types of diabetes, how to prevent or delay type 2 diabetes, and the serious health problems it can cause.'},
 {'url': 'https://www.mayoclinic.org/diseases-conditions/diabetes/symptoms-causes/syc-20371444',
  'content': "About Mayo Clinic\nMedical Professionals\nBusinesses\nMedical Professionals\nBusinesses\nStudents\nStudents\nResearchers\nInternational Patients\nCharitable Care & Financial Assistance\nFollow Mayo Clinic\nGet the Mayo Clinic app\n© 1998-2023 Mayo Foundation for Medical Education and Research (MFMER). Research\nEducation\nResearch\nEducation\nDiabetes\nOverview\nDiabetes mellitus refers to a group of diseases that affect how the body uses blood sugar (glucose). ErrorEmail field is required\nErrorInclude a valid email address\nTo provide you with the most relevant and helpful information, and u

In [ ]:
Router_Agent = Agent(
    role='Router',
    goal='Route user question to a vectorstore or web search',
    backstory=(
      "You are an expert at routing a user question to a vectorstore or web search."
      "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
      "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
    ),
    verbose=True,
    allow_delegation=False,
)

In [ ]:
Retriever_Agent = Agent(
    role="Retriever",
    goal="Use the information retrieved from the vectorstore to answer the question",
    backstory=(
        "You are an assistant for question-answering tasks."
        "Use the information present in the retrieved context to answer the question."
        "You have to provide a clear concise answer."
    ),
    verbose=True,
    allow_delegation=False,
)

In [ ]:
Grader_agent =  Agent(
    role='Answer Grader',
    goal='Filter out erroneous retrievals',
    backstory=(
      "You are a grader assessing relevance of a retrieved document to a user question."
      "If the document contains keywords related to the user question, grade it as relevant."
      "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
    ),
    verbose=True,
    allow_delegation=False,
)

In [ ]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
)

In [ ]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
)

In [ ]:
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [ ]:
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent = Retriever_Agent,
    context = [router_task],
)

In [ ]:
grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [ ]:
hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

answer_task = Task(
    description=("Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'websearch' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,
    #tools=[answer_grader_tool],
)

In [ ]:
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,
)

In [ ]:
inputs = {"question":"How we can implement AI in medical science"}

In [ ]:
result = rag_crew.kickoff(inputs=inputs)

# Agent: Router
## Task: Analyse the keywords in the question How we can implement AI in medical scienceBased on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


# Agent: Router
## Final Answer: 
websearch


# Agent: Retriever
## Task: Based on the response from the router task extract information for the question How we can implement AI in medical science with the help of the respective tool.Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'.Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'.


# Agent: Retriever
## Final Answer: 
Implementing AI in medical science involves various areas, including diagnostics, treatment recommendations, patient 

In [ ]:
print(result)

AI can be implemented in medical science in various ways, including improving diagnostic accuracy, personalizing treatment plans, predicting patient outcomes, automating administrative processes, and analyzing large datasets for research purposes. For instance, machine learning algorithms can assist radiologists in detecting anomalies in medical imaging, while natural language processing can help manage patient records. By leveraging AI, healthcare providers can enhance patient care, streamline operations, and drive innovative research advancements.
